# Basic usage

*`SciKeras`* is designed to maximize interoperability between `sklearn` and `Keras/TensorFlow`. The aim is to keep 99% of the flexibility of `Keras` while being able to leverage most features of `sklearn`. Below, we show the basic usage of `SciKeras` and how it can be combined with `sklearn`.

<table align="left"><td>
<a target="_blank" href="https://colab.research.google.com/github/adriangb/scikeras/blob/master/notebooks/Basic_Usage.ipyn">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>  
</td><td>
<a target="_blank" href="https://github.com/adriangb/scikeras/blob/master/notebooks/Basic_Usage.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

This notebook shows you how to use the basic functionality of `SciKeras`.

### Table of contents

* [Definition of the Keras Model](#Definition-of-the-keras-model)
* [Training a classifier](#Training-a-classifier-and-making-predictions)
  * [Dataset](#A-toy-binary-classification-task)
  * [Keras Model](#Definition-of-the-classification-model)
  * [Model training](#Training-the-neural-net-classifier)
  * [Inference](#Making-predictions,-classification)
* [Training a regressor](#Training-a-regressor)
  * [Dataset](#A-toy-regression-task)
  * [Keras Model](#Definition-of-the-regression-model)
  * [Model training](#Training-the-neural-net-regressor)
  * [Inference](#Making-predictions,-regression)
* [Saving and loading a model](#Saving-and-loading-a-model)
  * [Whole model](#Saving-the-whole-model)
  * [Only parameters](#Saving-only-the-model-parameters)
* [Usage with an sklearn Pipeline](#Usage-with-an-sklearn-Pipeline)
* [Callbacks](#Callbacks)
* [Grid search](#Usage-with-sklearn-GridSearchCV)
  * [Special prefixes](#Special-prefixes)
  * [Performing a grid search](#Performing-a-grid-search)

In [1]:
!python -m pip install git+https://github.com/adriangb/scikeras.git@master

  Cloning https://github.com/stsievert/scikeras.git (to revision data-processing-api) to /tmp/pip-req-build-2y7pm4r1
  Running command git clone -q https://github.com/stsievert/scikeras.git /tmp/pip-req-build-2y7pm4r1
  Running command git checkout -b data-processing-api --track origin/data-processing-api
  Switched to a new branch 'data-processing-api'
  Branch 'data-processing-api' set up to track remote branch 'data-processing-api' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scikeras: filename=scikeras-0.1.8-cp36-none-any.whl size=24422 sha256=1d1a707f8a28b5c63f5e37aae66ba799364feff509c09d0870848438d5c3aa77
  Stored in directory: /tmp/pip-ephem-wheel-cache-s3cc8ljb/wheels/03/39/e0/ab362dd5af349aa37278bfc764f064100df487ad11b80d046f
Successfully built scikeras


In [2]:
from scikeras.wrappers import KerasClassifier, KerasRegressor
from tensorflow import keras

## Training a classifier and making predictions

### A toy binary classification task

We load a toy classification task from `sklearn`.

In [3]:
import numpy as np
from sklearn.datasets import make_classification

In [4]:
X, y = make_classification(1000, 20, n_informative=10, random_state=0)

In [5]:
X.shape, y.shape, y.mean()

((1000, 20), (1000,), 0.5)

### Definition of the `Keras` classification `Model`

We define a vanilla neural network with.

Because we are dealing with 2 classes, the output layer can be constructed in
two different ways:
1. Single unit with a `"sigmoid"` nonlinearity. The loss must be `"binary_crossentropy"`.
2. Two units (one for each class) and a `"softmax"` nonlinearity. The loss must be `"sparse_categorical_crossentropy"`.

In this example, we choose the first option, which is what you would usually
do for binary classification. The second option is usually reserved for when
you have >2 classes.

In [6]:
from tensorflow import keras

In [7]:
def get_clf(meta, hidden_layer_sizes, dropout):
    n_features_in_ = meta["n_features_in_"]
    n_classes_ = meta["n_classes_"]
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(n_features_in_,)))
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    return model

### Defining and training the neural net classifier

We use `KerasClassifier` because we're dealing with a classifcation task. The first argument should be a callable returning a `Keras.Model`, in this case, `get_clf`. As additional arguments, we pass the number of loss function (required) and the optimizer, but the later is optional. We must also pass all of the arguments to `get_clf` as keyword arguments to `KerasClassifier` if they don't have a default value in `get_clf`. Note that if you do not pass an argument to `KerasClassifier`, it will not be avilable for hyperparameter tuning. Finally, we also pass `random_state=0` for reproducible results.

In [8]:
from scikeras.wrappers import KerasClassifier

In [9]:
clf = KerasClassifier(
    model=get_clf,
    loss="binary_crossentropy",
    optimizer="adam",
    hidden_layer_sizes=(100,),
    dropout=0.5,
)

As in `sklearn`, we call `fit` passing the input data `X` and the targets `y`.

In [10]:
clf.fit(X, y)

32/32 [==============================] - 0s 1ms/step - loss: 0.9342


KerasClassifier(
	model=<function get_clf at 0x7fceea7a92f0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=binary_crossentropy
	metrics=None
	batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	hidden_layer_sizes=(100,)
	dropout=0.5
)

Also, as in `sklearn`, you may call `predict` or `predict_proba` on the fitted model.

### Making predictions, classification

In [11]:
y_pred = clf.predict(X[:5])
y_pred

1/1 [==============================] - 0s 1ms/step


array([0, 0, 0, 0, 0])

In [12]:
y_proba = clf.predict_proba(X[:5])
y_proba

1/1 [==============================] - 0s 1ms/step


array([[0.5656425 , 0.4343575 ],
       [0.6093223 , 0.3906777 ],
       [0.5853057 , 0.4146943 ],
       [0.8011669 , 0.19883308],
       [0.6443313 , 0.35566872]], dtype=float32)

## Training a regressor

### A toy regression task

In [13]:
from sklearn.datasets import make_regression

In [14]:
X_regr, y_regr = make_regression(1000, 20, n_informative=10, random_state=0)

In [15]:
X_regr.shape, y_regr.shape, y_regr.min(), y_regr.max()

((1000, 20), (1000,), -649.0148244404172, 615.4505181286091)

### Definition of the `Keras` regression `Model`

Again, define a vanilla neural network. The main difference is that the output layer always has a single unit and does not apply any nonlinearity.

In [16]:
def get_reg(meta, hidden_layer_sizes, dropout):
    n_features_in_ = meta["n_features_in_"]
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(n_features_in_,)))
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1))
    return model

### Defining and training the neural net regressor

Training a regressor is almost the same as training a classifier. Mainly, we use `KerasRegressor` instead of `KerasClassifier` (this is the same terminology as in `sklearn`). We also change the loss function to `KerasRegressor.r_squared`. SciKeras provides this loss function because most of the `sklearn` ecosystem expects `R^2` as the loss function, but Keras does not have a default implementation.

In [17]:
from scikeras.wrappers import KerasRegressor

In [18]:
reg = KerasRegressor(
    model=get_reg,
    loss=KerasRegressor.r_squared,
    optimizer="adam",
    hidden_layer_sizes=(100,),
    dropout=0.5,
)

In [19]:
reg.fit(X_regr, y_regr)

32/32 [==============================] - 0s 1ms/step - loss: -0.0207


KerasRegressor(
	model=<function get_reg at 0x7fceea7a9488>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=<function KerasRegressor.r_squared at 0x7fceeb4e2620>
	metrics=None
	batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	hidden_layer_sizes=(100,)
	dropout=0.5
)

### Making predictions, regression

You may call `predict` or `predict_proba` on the fitted model. For regressions, both methods return the same value.

In [20]:
y_pred = reg.predict(X_regr[:5])
y_pred

1/1 [==============================] - 0s 1ms/step


array([-0.52893007,  0.87371266, -0.02472919,  0.14250527, -0.30587369])

## Saving and loading a model

Save and load either the whole model by using pickle, or use Keras' specialized save methods on the `KerasClassifier.model_` or `KerasRegressor.model_` attribute that is created after fitting. You will want to use Keras' model saving utilities if any of the following apply:
1. You wish to save only the weights or only the training configuration of your model.
2. You wish to share your model with collaborators. Pickle is a relatively unsafe protocol and it is not recommended to share or load pickle objects publically.
3. You care about performance, especially if doing in-memory serialization.

For more information, see Keras' [saving documentation](https://www.tensorflow.org/guide/keras/save_and_serialize).

### Saving the whole model

In [21]:
import pickle

In [22]:
bytes_model = pickle.dumps(reg)

In [23]:
new_reg = pickle.loads(bytes_model)
new_reg

KerasRegressor(
	model=<function get_reg at 0x7fceea7a9488>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=<function KerasRegressor.r_squared at 0x7fceeb4e2620>
	metrics=None
	batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	hidden_layer_sizes=(100,)
	dropout=0.5
)

### Saving using Keras' saving methods

This efficiently and safely saves the model to disk, including trained weights.


In [24]:
# Save to disk
reg.model_.save("/tmp/my_model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/my_model/assets


In [25]:
# Load the model back into memory
new_reg_model = keras.models.load_model("/tmp/my_model")
# Now we need to instantiate a new SciKeras object with this model
reg = KerasRegressor(
    new_reg_model,
    loss=KerasRegressor.r_squared,
    optimizer="adam",
    hidden_layer_sizes=(100,),
    dropout=0.5,
    random_state=0
)

## Usage with an `sklearn Pipeline`

It is possible to put the `KerasClassifier` inside an `sklearn Pipeline`, as you would with any `sklearn` classifier.

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [27]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('clf', clf),
])

In [28]:
pipe.fit(X, y)

32/32 [==============================] - 0s 1ms/step - loss: 0.7180


Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 KerasClassifier(batch_size=None, build_fn=None, callbacks=None, dropout=0.5, epochs=1, hidden_layer_sizes=(100,), loss='binary_crossentropy', metrics=None, model=<function get_clf at 0x7fceea7a92f0>, optimizer='adam', random_state=None, run_eagerly=False, shuffle=True, validation_split=0.0, verbose=1, warm_start=False))],
         verbose=False)

In [29]:
y_proba = pipe.predict_proba(X[:5])
y_proba

1/1 [==============================] - 0s 2ms/step


array([[0.25762576, 0.74237424],
       [0.5406221 , 0.45937788],
       [0.5918224 , 0.40817758],
       [0.6329869 , 0.3670131 ],
       [0.5921637 , 0.40783632]], dtype=float32)

To save the whole pipeline, including the Keras model, use `pickle`.

## Callbacks

Adding a new callback to the model is straightforward. Below we show how to add an `EarlyStopping` callback to prevent overfitting.

In [30]:
es = keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', patience=200, verbose=1)

We now generate a toy dataset using `sklearn.datasets.make_moons`. This dataset was chosen specifically to trigger early stopping.

In [31]:
from sklearn.datasets import make_moons

In [32]:
X, y = make_moons(n_samples=100, noise=0.2, random_state=0)
X.shape, y.shape

((100, 2), (100,))

We will first check fitting without the callback and then with. We will compare the training time and final accuracy.

In [33]:
import time

In [34]:
# First test without the callback
clf = KerasClassifier(
    model=get_clf,
    loss="binary_crossentropy",
    optimizer="adam",
    hidden_layer_sizes=(500,),
    dropout=0.5,
    metrics=["binary_accuracy"],
    fit__validation_split=0.2,
    epochs=4000,
    verbose=False,
)
start = time.time()
clf.fit(X, y)
print(f"Training time: {time.time() - start}")
print(f"Final accuracy: {clf.history_['val_binary_accuracy'][-1][-1]}")  # get last value of last fit/partial_fit call

Training time: 71.7183313369751
Final accuracy: 1.0


In [35]:
# Test with the callback
clf = KerasClassifier(
    model=get_clf,
    loss="binary_crossentropy",
    optimizer="adam",
    hidden_layer_sizes=(500,),
    dropout=0.5,
    metrics=["binary_accuracy"],
    fit__validation_split=0.2,
    epochs=4000,
    verbose=False,
    callbacks=[es]
)
start = time.time()
clf.fit(X, y)
print(f"Training time: {time.time() - start}")
print(f"Final accuracy: {clf.history_['val_binary_accuracy'][-1][-1]}")  # get last value of last fit/partial_fit call

Epoch 00217: early stopping
Training time: 4.529699802398682
Final accuracy: 1.0


For information on how to write custom callbacks, have a look at the 

---

[Advanced_Usage](https://nbviewer.jupyter.org/github/adriangb/scikeras/blob/master/notebooks/Advanced_Usage.ipynb) notebook.

## Usage with sklearn `GridSearchCV`

### Special prefixes

SciKeras allows to direct access to all parameters passed to the wrapper constructors, including deeply nested routed parameters. This allows tunning of
paramters like `hidden_layer_sizes` as well as `optimizer__learning_rate`.

This is exactly the same logic that allows to access estimator parameters in `sklearn Pipeline`s and `FeatureUnion`s.

This feature is useful in several ways. For one, it allows to set those parameters in the model definition. Furthermore, it allows you to set parameters in an `sklearn GridSearchCV` as shown below.

To differentiate paramters like `callbacks` which are accepted by both `tf.keras.Model.fit` and `tf.keras.Model.predict` you can add a `fit__` or `predict__` routing suffix respectively. Similar, the `model__` prefix may be used to specify that a paramter is destined only for `get_clf`/`get_reg` (or whatever callable you pass as your `model` argument).

For more information on parameter routing with special prefixes, see the [Advanced Usage Docs](https://scikeras.org.readthedocs.build/en/latest/advanced.html#routed-parameters)

### Performing a grid search

Below we show how to perform a grid search over the learning rate (`optimizer__lr`), the model's number of hidden layers (`model__hidden_layer_sizes`), the model's dropout rate (`model__dropout`).

In [36]:
from sklearn.model_selection import GridSearchCV

In [37]:
clf = KerasClassifier(
    model=get_clf,
    loss="binary_crossentropy",
    optimizer="adam",
    optimizer__lr=0.1,
    model__hidden_layer_sizes=(100,),
    model__dropout=0.5,
    verbose=False,
)

*Note*: We set the verbosity level to zero (`verbose=False`) to prevent too much print output from being shown.

In [38]:
params = {
    'optimizer__lr': [0.05, 0.1],
    'model__hidden_layer_sizes': [(100, ), (50, 50, ), (33, 33, 33, )],
    'model__dropout': [0, 0.5],
}

In [39]:
gs = GridSearchCV(clf, params, refit=False, cv=3, scoring='accuracy', verbose=2)

In [40]:
gs.fit(X, y)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] model__dropout=0, model__hidden_layer_sizes=(100,), optimizer__lr=0.05 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  model__dropout=0, model__hidden_layer_sizes=(100,), optimizer__lr=0.05, total=   0.7s
[CV] model__dropout=0, model__hidden_layer_sizes=(100,), optimizer__lr=0.05 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV]  model__dropout=0, model__hidden_layer_sizes=(100,), optimizer__lr=0.05, total=   0.5s
[CV] model__dropout=0, model__hidden_layer_sizes=(100,), optimizer__lr=0.05 
[CV]  model__dropout=0, model__hidden_layer_sizes=(100,), optimizer__lr=0.05, total=   0.5s
[CV] model__dropout=0, model__hidden_layer_sizes=(100,), optimizer__lr=0.1 
[CV]  model__dropout=0, model__hidden_layer_sizes=(100,), optimizer__lr=0.1, total=   0.5s
[CV] model__dropout=0, model__hidden_layer_sizes=(100,), optimizer__lr=0.1 
[CV]  model__dropout=0, model__hidden_layer_sizes=(100,), optimizer__lr=0.1, total=   0.5s
[CV] model__dropout=0, model__hidden_layer_sizes=(100,), optimizer__lr=0.1 
[CV]  model__dropout=0, model__hidden_layer_sizes=(100,), optimizer__lr=0.1, total=   0.5s
[CV] model__dropout=0, model__hidden_layer_sizes=(50, 50), optimizer__lr=0.05 
[CV]  model__dropout=0, model__hidden_layer_sizes=(50, 50), optimizer__lr=0.05, total=   0.6s
[CV] model__dropout=0, model__hidden_layer_sizes=(50, 50), optimi

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:   21.0s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=KerasClassifier(batch_size=None, build_fn=None, callbacks=None, epochs=1, loss='binary_crossentropy', metrics=None, model=<function get_clf at 0x7fceea7a92f0>, model__dropout=0.5, model__hidden_layer_sizes=(100,), optimizer='adam', optimizer__lr=0.1, random_state=None, run_eagerly=False, shuffle=True, validation_split=0.0, verbose=False, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'model__dropout': [0, 0.5],
                         'model__hidden_layer_sizes': [(100,), (50, 50),
                                                       (33, 33, 33)],
                         'optimizer__lr': [0.05, 0.1]},
             pre_dispatch='2*n_jobs', refit=False, return_train_score=False,
             scoring='accuracy', verbose=2)

In [41]:
print(gs.best_score_, gs.best_params_)

0.8900772430184194 {'model__dropout': 0, 'model__hidden_layer_sizes': (50, 50), 'optimizer__lr': 0.1}


Of course, we could further nest the `KerasClassifier` within an `sklearn Pipeline`, in which case we just prefix the parameter by the name of the net (e.g. `clf__model__hidden_layer_sizes`).